In [584]:
import random
import string
import binascii
from collections import deque
#############################################################################################################

#########S-Box used for key expansion and encrytion####################################################
Sbox=[['63','7c','77','7b','f2','6b','6f','c5','30','01','67','2b','fe','d7','ab','76'],
      ['ca','82','c9','7d','fa','59','47','f0','ad','d4','a2','af','9c','a4','72','c0'],
      ['b7','fd','93','26','36','3f','f7','cc','34','a5','e5','f1','71','d8','31','15'],
      ['04','c7','23','c3','18','96','05','9a','07','12','80','e2','eb','27','b2','75'],
      ['09','83','2c','1a','1b','6e','5a','a0','52','3b','d6','b3','29','e3','2f','84'],
      ['53','d1','00','ed','20','fc','b1','5b','6a','cb','be','39','4a','4c','58','cf'],
      ['d0','ef','aa','fb','43','4d','33','85','45','f9','02','7f','50','3c','9f','a8'],
      ['51','a3','40','8f','92','9d','38','f5','bc','b6','da','21','10','ff','f3','d2'],
      ['cd','0c','13','ec','5f','97','44','17','c4','a7','7e','3d','64','5d','19','73'],
      ['60','81','4f','dc','22','2a','90','88','46','ee','b8','14','de','5e','0b','db'],
      ['e0','32','3a','0a','49','06','24','5c','c2','d3','ac','62','91','95','e4','79'],
      ['e7','c8','37','6d','8d','d5','4e','a9','6c','56','f4','ea','65','7a','ae','08'],
      ['ba','78','25','2e','1c','a6','b4','c6','e8','dd','74','1f','4b','bd','8b','8a'],
      ['70','3e','b5','66','48','03','f6','0e','61','35','57','b9','86','c1','1d','9e'],
      ['e1','f8','98','11','69','d9','8e','94','9b','1e','87','e9','ce','55','28','df'],
      ['8c','a1','89','0d','bf','e6','42','68','41','99','2d','0f','b0','54','bb','16']]
#########inverse S-Box used for key expansion and encrytion####################################################
S_Inverse_box=[['52','09','6a','d5','30','36','a5','38','bf','40','a3','9e','81','f3','d7','fb'],
               ['7c','e3','39','82','9b','2f','ff','87','34','8e','43','44','c4','de','e9','cb'],
               ['54','7b','94','32','a6','c2','23','3d','ee','4c','95','0b','42','fa','c3','4e'],
               ['08','2e','a1','66','28','d9','24','b2','76','5b','a2','49','6d','8b','d1','25'],
               ['72','f8','f6','64','86','68','98','16','d4','a4','5c','cc','5d','65','b6','92'],
               ['6c','70','48','50','fd','ed','b9','da','5e','15','46','57','a7','8d','9d','84'],
               ['90','d8','ab','00','8c','bc','d3','0a','f7','e4','58','05','b8','b3','45','06'],
               ['d0','2c','1e','8f','ca','3f','0f','02','c1','af','bd','03','01','13','8a','6b'],
               ['3a','91','11','41','4f','67','dc','ea','97','f2','cf','ce','f0','b4','e6','73'],
               ['96','ac','74','22','e7','ad','35','85','e2','f9','37','e8','1c','75','df','6e'],
               ['47','f1','1a','71','1d','29','c5','89','6f','b7','62','0e','aa','18','be','1b'],
               ['fc','56','3e','4b','c6','d2','79','20','9a','db','c0','fe','78','cd','5a','f4'],
               ['1f','dd','a8','33','88','07','c7','31','b1','12','10','59','27','80','ec','5f'],
               ['60','51','7f','a9','19','b5','4a','0d','2d','e5','7a','9f','93','c9','9c','ef'],
               ['a0','e0','3b','4d','ae','2a','f5','b0','c8','eb','bb','3c','83','53','99','61'],
               ['17','2b','04','7e','ba','77','d6','26','e1','69','14','63','55','21','0c','7d']]
#########function for generating round constant for particular round##########################################
def generate_rcon(rnd):
    rcon = 0x8d
    for i in range(0, rnd):
        rcon = ((rcon << 1) ^ (0x11b & - (rcon >> 7))) & 0xff
    return rcon
#########function to rotate an array left by n################################################################
def rotate_left(lst, n):
    return lst[n:] + lst[:n]
#########function to rotate an array right by n################################################################
def rotate_right(lst, n):
    ll = len(lst)
    return lst[(ll-n):] + lst[:(ll-n)]
#########function to generate a 128 bit key ##################################################################
def generate_Key(customkey = ""):
    if customkey != "":
        if len(customkey) == 16:
            return customkey
        else:
            print("custom key is of wrong length using random key")
            lettersAndDigits = string.ascii_letters + string.digits
            key = ''.join(random.choice(lettersAndDigits) for i in range(16))
            return key
    else:
        lettersAndDigits = string.ascii_letters + string.digits
        key = ''.join(random.choice(lettersAndDigits) for i in range(16))
        return key
#########function used in key expansion on w3###################################################################
def g(w3,rnd):
######left rotate w3 once###################        
    w3 = rotate_left(w3,1)
######substitue with s-box##################
    for i in range(0,4,1):
        if len(w3[i]) ==1:
            temp=w3[i]
            w3[i]='0' + temp
    for i in range(0,4,1):
        w3[i] =  Sbox[int(w3[i][0],16)] [int(w3[i][1],16)]
######adding round constant#################
    w3[0] = hex(int(w3[0],16)^generate_rcon(rnd))[2:]
######returning w3##########################    
    return w3
########function for generating round keys#########################################################    
def gererate_round_keys(key):
    round_keys=[]    
    key_matrix = [[0] * 4 for i in range(4)]
    ind=0
    for i in range(0,4,1):
        for j in range(0,4,1):
            temp=ord(key[ind])
            temp=hex(temp)
            key_matrix[j][i] = temp[2] + temp[3]
            ind=ind+1
    #####generating w0-w3 from key matrix ##############
    w0=[]
    w1=[]
    w2=[]
    w3=[]
    for i in range(0,4,1):
        w0.append(key_matrix[i][0])
        w1.append(key_matrix[i][1])
        w2.append(key_matrix[i][2])
        w3.append(key_matrix[i][3]) 
    rk = w0+w1+w2+w3
    round_keys.append(rk)
    ######generating all other round keys ##############
    for i in range(10):
        w4=[]
        w5=[]
        w6=[]
        w7=[]
        gw3 = g(w3,i+1)
        for i in range(0,4,1):
            w4.append(hex(int(gw3[i],16) ^ int(w0[i],16))[2:])
            w5.append(hex(int(w4[i],16) ^ int(w1[i],16))[2:])
            w6.append(hex(int(w5[i],16) ^ int(w2[i],16))[2:])
            w7.append(hex(int(w6[i],16) ^ int(w3[i],16))[2:])
        w0 = w4
        w1 = w5
        w2 = w6
        w3 = w7
        rk = w0+w1+w2+w3
        round_keys.append(rk)
    return round_keys
########convert key to matrix###############################################################################
def key_to_matrix(round_key):
    key_matrix = [[0] * 4 for i in range(4)]
    ind=0
    for i in range(0,4,1):
        for j in range(0,4,1):
            key_matrix[j][i] = round_key[ind]
            ind=ind+1
    return key_matrix  
########Substitue bytes in state matrix using S-box#########################################################
def sub_bytes(state_matrix):
    for i in range(4):
        for j in range(4):
            if len(state_matrix[i][j]) ==1:
                temp=state_matrix[i][j]
                state_matrix[i][j]='0' + temp
    for i in range(4):
        for j in range(4):
            state_matrix[i][j] = Sbox[int(state_matrix[i][j][0],16)][int(state_matrix[i][j][1],16)]
    return state_matrix
########Inverse Substitue bytes in state matrix using S-box#########################################################
def inv_sub_bytes(state_matrix):
    for i in range(4):
        for j in range(4):
            if len(state_matrix[i][j]) ==1:
                temp=state_matrix[i][j]
                state_matrix[i][j]='0' + temp
    for i in range(4):
        for j in range(4):
            state_matrix[i][j] = S_Inverse_box[int(state_matrix[i][j][0],16)][int(state_matrix[i][j][1],16)]
    return state_matrix
########shift rows of state matrix##########################################################################
def shift_rows(state_matrix):
    for i in range(4):
        state_matrix[i] =rotate_left(state_matrix[i],i)
    return state_matrix
########inverse shift rows of state matrix##########################################################################
def inv_shift_rows(state_matrix):
    for i in range(4):
        state_matrix[i] =rotate_right(state_matrix[i],i)
    return state_matrix
########functions for mixing columns########################################################################
def gfMultiply(cfm,smv):
    r = 0
    if cfm=='01':
        return int(smv,16)
    elif cfm=='02':
        r = (int(smv,16)<<1)
        if r>0xFF:
            r = r^0x11b
    else:
        r = int(smv,16)<<1
        if r>0xFF:
            r = r^0x11b
        r = r^int(smv,16)
    return r
def MixColumns(state_matrix):
    result_matrix = [[0] * 4 for i in range(4)]
    A = [['02','03','01','01'],
         ['01','02','03','01'],
         ['01','01','02','03'],
         ['03','01','01','02']]
    ind = 0
    for i in range(4):
        for j in range(4):
            temp=0
            for k in range(4):
                    temp = gfMultiply(A[i][k],state_matrix[k][j])
                    result_matrix[i][j] ^= temp
                    ind += 1
    for i in range(4):
        for j in range(4):
            state_matrix[i][j] = hex(result_matrix[i][j])[2:]
    return state_matrix
########functions for inverse mixing columns########################################################################
def mul_by_02(num):
    if int(num,16) < 128:
        res = (int(num,16) << 1)
    else:
        res = (int(num,16) << 1) ^ 0x1b

    return hex(res % 256)[2:]
def inv_gfMultiply(cfm,num):
    
    if cfm == '09':
        return int(mul_by_02(mul_by_02(mul_by_02(num))),16) ^ int(num,16)
    elif cfm == '0b':
        return int(mul_by_02(mul_by_02(mul_by_02(num))),16) ^ int(mul_by_02(num),16) ^ int(num,16)
    elif cfm == '0d':
        return int(mul_by_02(mul_by_02(mul_by_02(num))),16) ^ int(mul_by_02(mul_by_02(num)),16) ^ int(num,16)
    elif cfm == "0e":
        return int(mul_by_02(mul_by_02(mul_by_02(num))),16) ^ int(mul_by_02(mul_by_02(num)),16) ^ int(mul_by_02(num),16)

def inv_MixColumns(state_matrix):
    result_matrix = [[0] * 4 for i in range(4)]
    inv_cm = [['0e','0b','0d','09'],
              ['09','0e','0b','0d'],
              ['0d','09','0e','0b'],
              ['0b','0d','09','0e']]
    for i in range(4):
        for j in range(4):
            temp=0
            for k in range(4):
                    temp = inv_gfMultiply(inv_cm[i][k],state_matrix[k][j])
                    result_matrix[i][j] ^= temp
    for i in range(4):
        for j in range(4):
            state_matrix[i][j] = hex(result_matrix[i][j])[2:]
    return state_matrix
########function for encryption with all round keys#########################################################    
def encrypt(Plain_text,round_keys):
    state_matrix = [[0] * 4 for i in range(4)]
    text_matrix = [[0] * 4 for i in range(4)]
########converting plain text to matrix and hex########  
    ind=0
    for i in range(0,4,1):
        for j in range(0,4,1):
            temp=ord(Plain_text[ind])
            temp=hex(temp)
            text_matrix[j][i] = temp[2] + temp[3]
            ind=ind+1
########taking round 0 key and performing key whitening########
    key_matrix = key_to_matrix(round_keys[0])
    for i in range(4):
        for j in range(4):
            state_matrix[i][j] = hex(int(text_matrix[i][j],16) ^ int(key_matrix[i][j],16))[2:]
    print(state_matrix, " key whitening ")
########performing round 1-9 encryption#######
    for rnd in range(9):
        state_matrix = sub_bytes(state_matrix)
        print(state_matrix, " sub bytes round: ",rnd+1)
        state_matrix = shift_rows(state_matrix)
        print(state_matrix," shift rows round: ",rnd+1)
        state_matrix = MixColumns(state_matrix)
        print(state_matrix," mix cols round: ",rnd+1)
        key_matrix = key_to_matrix(round_keys[rnd+1])
        for i in range(4):
            for j in range(4):
                state_matrix[i][j] = hex(int(state_matrix[i][j],16) ^ int(key_matrix[i][j],16))[2:]
        print(state_matrix," encrpted round: ",rnd+1)
########performing round 10 encryption without mixing columns#######
    state_matrix = sub_bytes(state_matrix)
    state_matrix = shift_rows(state_matrix)
    key_matrix = key_to_matrix(round_keys[10])
    for i in range(0,4,1):
        for j in range(0,4,1):
            state_matrix[i][j] = hex(int(state_matrix[i][j],16) ^ int(key_matrix[i][j],16))[2:]
    print(state_matrix," encrypted round: 10")
#######converting state matrix to list to return cipher text#######
    encrypted_text =[]
    for i in range(4):
        for j in range(4):
            encrypted_text.append(state_matrix[j][i])
    return(encrypted_text)
########function for decryption with all round keys#########################################################    
def decrypt(cipher_text,round_keys):
    state_matrix = [[0] * 4 for i in range(4)]
    text_matrix = [[0] * 4 for i in range(4)]
########converting cipher text to matrix and hex######## 
    ind = 0
    for i in range(0,4,1):
        for j in range(0,4,1):
            text_matrix[j][i] = cipher_text[ind]  
            ind = ind+1
    print(text_matrix," cipher matrix ")
########taking round 10 key and performing key whitening########
    key_matrix = key_to_matrix(round_keys[10])
    for i in range(4):
        for j in range(4):
            state_matrix[i][j] = hex(int(text_matrix[i][j],16) ^ int(key_matrix[i][j],16))[2:]
    print(state_matrix, " key whitening inverse ")
########performing round 9-1 decryption#######
    for rnd in range(9):
        state_matrix = inv_shift_rows(state_matrix)
        print(state_matrix," I_shift rows round:",9-rnd)
        state_matrix = inv_sub_bytes(state_matrix)
        print(state_matrix, " I_sub bytes round:",9-rnd)
        key_matrix = key_to_matrix(round_keys[9-rnd])
        for i in range(4):
            for j in range(4):
                state_matrix[i][j] = hex(int(state_matrix[i][j],16) ^ int(key_matrix[i][j],16))[2:]
        print(state_matrix," decrpted round: ",9-rnd)
        state_matrix = inv_MixColumns(state_matrix)
        print(state_matrix," I_mix cols round: ",9-rnd)
########performing round 10 encryption without inverse mixing columns#######
    state_matrix = inv_shift_rows(state_matrix)
    print(state_matrix," I_shift rows round:",0)
    state_matrix = inv_sub_bytes(state_matrix)
    print(state_matrix, " I_sub bytes round:",0)
    key_matrix = key_to_matrix(round_keys[0])
    for i in range(0,4,1):
        for j in range(0,4,1):
            state_matrix[i][j] = hex(int(state_matrix[i][j],16) ^ int(key_matrix[i][j],16))[2:]
    print(state_matrix," decrypted round: 0")
#######converting state matrix to list to return decrypted text#######
    decrypted_text =[]
    for i in range(4):
        for j in range(4):
            decrypted_text.append(state_matrix[j][i])
    return(decrypted_text)

In [585]:
key = generate_Key("Thats my Kung Fu")
print("key being used: ",key)
round_keys = gererate_round_keys(key)
for i in range(len(round_keys)):
    print("round key for round : ",i,round_keys[i])

key being used:  Thats my Kung Fu
round key for round :  0 ['54', '68', '61', '74', '73', '20', '6d', '79', '20', '4b', '75', '6e', '67', '20', '46', '75']
round key for round :  1 ['e2', '32', 'fc', 'f1', '91', '12', '91', '88', 'b1', '59', 'e4', 'e6', 'd6', '79', 'a2', '93']
round key for round :  2 ['56', '8', '20', '7', 'c7', '1a', 'b1', '8f', '76', '43', '55', '69', 'a0', '3a', 'f7', 'fa']
round key for round :  3 ['d2', '60', 'd', 'e7', '15', '7a', 'bc', '68', '63', '39', 'e9', '1', 'c3', '3', '1e', 'fb']
round key for round :  4 ['a1', '12', '2', 'c9', 'b4', '68', 'be', 'a1', 'd7', '51', '57', 'a0', '14', '52', '49', '5b']
round key for round :  5 ['b1', '29', '3b', '33', '5', '41', '85', '92', 'd2', '10', 'd2', '32', 'c6', '42', '9b', '69']
round key for round :  6 ['bd', '3d', 'c2', '87', 'b8', '7c', '47', '15', '6a', '6c', '95', '27', 'ac', '2e', 'e', '4e']
round key for round :  7 ['cc', '96', 'ed', '16', '74', 'ea', 'aa', '3', '1e', '86', '3f', '24', 'b2', 'a8', '31', '6a']

In [586]:
Plain_text = "Two One Nine Two"
cipher_text = encrypt(Plain_text,round_keys)
print("cipher text after all 10 rounds: \n",cipher_text)

[['0', '3c', '6e', '47'], ['1f', '4e', '22', '74'], ['e', '8', '1b', '31'], ['54', '59', 'b', '1a']]  key whitening 
[['63', 'eb', '9f', 'a0'], ['c0', '2f', '93', '92'], ['ab', '30', 'af', 'c7'], ['20', 'cb', '2b', 'a2']]  sub bytes round:  1
[['63', 'eb', '9f', 'a0'], ['2f', '93', '92', 'c0'], ['af', 'c7', 'ab', '30'], ['a2', '20', 'cb', '2b']]  shift rows round:  1
[['ba', '84', 'e8', '1b'], ['75', 'a4', '8d', '40'], ['f4', '8d', '6', '7d'], ['7a', '32', 'e', '5d']]  mix cols round:  1
[['58', '15', '59', 'cd'], ['47', 'b6', 'd4', '39'], ['8', '1c', 'e2', 'df'], ['8b', 'ba', 'e8', 'ce']]  encrpted round:  1
[['6a', '59', 'cb', 'bd'], ['a0', '4e', '48', '12'], ['30', '9c', '98', '9e'], ['3d', 'f4', '9b', '8b']]  sub bytes round:  2
[['6a', '59', 'cb', 'bd'], ['4e', '48', '12', 'a0'], ['98', '9e', '30', '9c'], ['8b', '3d', 'f4', '9b']]  shift rows round:  2
[['15', 'c9', '7f', '9d'], ['ce', '4d', '4b', 'c2'], ['89', '71', 'be', '88'], ['65', '47', '97', 'cd']]  mix cols round:  2
[['43

In [587]:
decrypted_text = decrypt(cipher_text,round_keys)
ch=[]
for i in range(len(decrypted_text)):
    ch.append(chr(int(decrypted_text[i],16)))
print(ch)

[['29', '57', '40', '1a'], ['c3', '14', '22', '2'], ['50', '20', '99', 'd7'], ['5f', 'f6', 'b3', '3a']]  cipher matrix 
[['1', '3a', '8c', '21'], ['3e', 'b0', 'e2', '33'], ['8e', '4', '3d', 'b8'], ['a7', 'bc', '4d', '1c']]  key whitening inverse 
[['1', '3a', '8c', '21'], ['33', '3e', 'b0', 'e2'], ['3d', 'b8', '8e', '4'], ['bc', '4d', '1c', 'a7']]  I_shift rows round: 9
[['09', 'a2', 'f0', '7b'], ['66', 'd1', 'fc', '3b'], ['8b', '9a', 'e6', '30'], ['78', '65', 'c4', '89']]  I_sub bytes round: 9
[['b6', 'e7', '51', '8c'], ['84', '88', '98', 'ca'], ['34', '60', '66', 'fb'], ['e8', 'd7', '70', '51']]  decrpted round:  9
[['33', '51', '79', 'a'], ['8b', '66', '8f', '3f'], ['76', '7d', 'eb', 'be'], ['20', '92', 'c2', '67']]  I_mix cols round:  9
[['33', '51', '79', 'a'], ['3f', '8b', '66', '8f'], ['eb', 'be', '76', '7d'], ['92', 'c2', '67', '20']]  I_shift rows round: 8
[['66', '70', 'af', 'a3'], ['25', 'ce', 'd3', '73'], ['3c', '5a', '0f', '13'], ['74', 'a8', '0a', '54']]  I_sub bytes roun

In [590]:
def pad(text):
    pad_len = 16 - (len(text) % 16)
    pad_string = "0" * pad_len
    return text + pad_string
def fix_size(text):
    if len(text)%16 == 0:
        return [text[i:i+16] for i in range(0, len(text), 16)]
    else:
        text = pad(text)
        return [text[i:i+16] for i in range(0, len(text), 16)]

In [600]:
############################## user inputs implemented ############################

text = input("Enter a plain text to be incrypted: ")
user_key = input("Enter a 128 bit key: ")
key = generate_Key(user_key)
print("key being used: ",key)
round_keys = gererate_round_keys(key)
for i in range(len(round_keys)):
    print("round key for round : ",i,round_keys[i])
text = fix_size(text)
all_cipher_texts = []
for i in range(len(text)):
    cipher_text = encrypt(text[i],round_keys)
    all_cipher_texts.append(cipher_text)
for i in range(len(all_cipher_texts)):
    print(all_cipher_texts[i]," ",i)
all_decipher_texts = []
for i in range(len(text)):
    decipher_text = decrypt(all_cipher_texts[i],round_keys)
    all_decipher_texts.append(decipher_text)
for i in range(len(all_decipher_texts)):
    print(all_decipher_texts[i]," ",i)
ch=[]
for i in range(len(all_decipher_texts)):
    for j in range(len(all_decipher_texts[i])):
        ch.append(chr(int(all_decipher_texts[i][j],16)))
strng=""
for i in ch:
    strng +=i
print(string)    

Enter a plain text to be incrypted: random text to see if this works correctly
Enter a 128 bit key: hope this key ok
key being used:  hope this key ok
round key for round :  0 ['68', '6f', '70', '65', '20', '74', '68', '69', '73', '20', '6b', '65', '79', '20', '6f', '6b']
round key for round :  1 ['de', 'c7', 'f', 'd3', 'fe', 'b3', '67', 'ba', '8d', '93', 'c', 'df', 'f4', 'b3', '63', 'b4']
round key for round :  2 ['b1', '3c', '82', '6c', '4f', '8f', 'e5', 'd6', 'c2', '1c', 'e9', '9', '36', 'af', '8a', 'bd']
round key for round :  3 ['cc', '42', 'f8', '69', '83', 'cd', '1d', 'bf', '41', 'd1', 'f4', 'b6', '77', '7e', '7e', 'b']
round key for round :  4 ['37', 'b1', 'd3', '9c', 'b4', '7c', 'ce', '23', 'f5', 'ad', '3a', '95', '82', 'd3', '44', '9e']
round key for round :  5 ['41', 'aa', 'd8', '8f', 'f5', 'd6', '16', 'ac', '0', '7b', '2c', '39', '82', 'a8', '68', 'a7']
round key for round :  6 ['a3', 'ef', '84', '9c', '56', '39', '92', '30', '56', '42', 'be', '9', 'd4', 'ea', 'd6', 'ae']
r